# მოდელის ამოცანის დასმა
<span style="font-size: 16px;">დიეტა უამრავი ადამიანებისთვის ცხოვრების მნიშვნელოვანი ნაწილია; იქნება ეს ჯანმრთელობის გაუმჯობესების, შეხედულების შენარჩუნებისა თუ სხვა მიზეზების გამო. ზოგიერთ შემთხვევაში კი დიეტის მთავარ მიზანს საკვების გარკვეული ნივთიერების კონსისტენტურად მიღება წარმოადგენს. მოცემული მონაცემებიდან ჩვენ გვინდა, რომ ავაგოთ მოდელი, რომელიც მომხმარებელს შესთავაზებს, თუ რა საკვების მიღება არის მისთვის პრიორიტეტი, რათა სწორად მიჰყვეს მის დიეტას და მისთვის სასურველ შედეგს მიაღწიოს.</span>

<span style="font-size: 16px;">მიზნის მისაღწევად თავდაპირველად დავამუშავებთ მონაცემებს და გამოვიყენებთ ისეთ დიეტებს, რომელთაც მომხმარებლები დიდი სიზუსტით და დარღვევის გარეშე მიჰყვებოდნენ, მიღებული დიეტების შემადგენელ საკვებს კი შემდეგ მომხმარებლისთვის რეკომენდაციისთვის გამოვიყენებთ. მომხმარებელი გადმოგვცემს მონაცემებს, თუ რა არის მისთვის სასურველი/მისაღწევი საკვების ნივთიერებების შემადგენლობა, და თუ დღემდე როგორ მიჰყვებოდა თავის დიეტას, რისი დამუშავებითაც ჩვენი მოდელი ნახავს, თუ სავარაუდოდ რა შემცველობის საკვებს მიიღებს იგი შემდეგ დღეს, და გადაწყვეტს ნავარაუდევი საკვები კარგი არის თუ არა მომხმარებლისთვის და მისი დიეტისთვის, რის შემდეგაც მომხმარებელს მიღებულ შედეგს შეატყობინებს.</span>

# მონაცემების გაანალიზება

<span style="font-size: 16px;">**ამოცანის გადაჭრის პირველ ეტაპზე გავარჩიოთ გადმოცემული მონაცემები**</span>

<span style="font-size: 16px;">თავდაპირველად დავაიმპორტოთ საჭირო ბიბლიოთეკები და შევინახოთ მონაცემები `train` ცვლადში.</span>

In [6]:
# Import libraries
import numpy as np
import pandas as pd
# We will need json as the given data and users' diet days are represented as JSON objects 
import json

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# Store data
train = pd.read_csv('mfp-diaries.tsv', sep='\t')

<span style="font-size: 16px;">ვნახოთ, თუ როგორ გამოიყურება და რა სახის ფორმატში გვაქვს გადმოცემული მონაცემები</span>

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587186 entries, 0 to 587185
Data columns (total 4 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

<span style="font-size: 16px;">მონაცემებზე კარგად დაკვირვებით ვხედავთ, რომ ჩვენი მონაცემები შედგება **მომხმარებლის იდენტიფიკატორით**, **თარიღით**, მიმდინარე დღეს **მიღებული საკვების ნივთიერებების რაოდენობითა** და მისი მიმდინარე დღის **სასურველი ნივთიერებების მიღების რაოდენობა (მიზანი)**. მონაცემების უკეთ დასანახად/გასანაწილებლად ვნახოთ პირველი მომხმარებლის პირველი დღის ჩანაწერი:</span>

In [20]:
def get_details_for_index(data, row_index):
    """This function returns details of user data for given row index"""
    # Id of the first user
    user_id = data.iloc[0, 0]
    # Date of the first user's first day of the diet
    date = data.iloc[0, 1]
    # User's intake of neutrients for the day
    food_entries = json.loads(data.iloc[0, 2])
    # User's goal intake of neutrients for the day
    intake_goal = json.loads(data.iloc[0,3])
    
    return (user_id, date, food_entries, intake_goal)

In [21]:
details = get_details_for_index(train, 0)

print('\n------------- User ' + str(details[0]) + ' Date ' + str(details[1]) + ' -------------\n')
print('------------- Food Entries -------------')
print(details[2])
print('\n------------- Goal -------------')
print(details[3])


------------- User 1 Date 2014-09-15 -------------

------------- Food Entries -------------
[{'meal': 'MY food', 'dishes': [{'nutritions': [{'name': 'Calories', 'value': '412'}, {'name': 'Carbs', 'value': '29'}, {'name': 'Fat', 'value': '24'}, {'name': 'Protein', 'value': '21'}, {'name': 'Sodium', 'value': '258'}, {'name': 'Sugar', 'value': '29'}], 'name': 'my - McDonalds Espresso Pronto® Flat White, 2 TALL'}, {'nutritions': [{'name': 'Calories', 'value': '176'}, {'name': 'Carbs', 'value': '33'}, {'name': 'Fat', 'value': '1'}, {'name': 'Protein', 'value': '5'}, {'name': 'Sodium', 'value': '195'}, {'name': 'Sugar', 'value': '0'}], 'name': 'Uncle Tobys Australia - Vita Brits, 3 Biscuits 33.3g'}, {'nutritions': [{'name': 'Calories', 'value': '342'}, {'name': 'Carbs', 'value': '34'}, {'name': 'Fat', 'value': '12'}, {'name': 'Protein', 'value': '24'}, {'name': 'Sodium', 'value': '402'}, {'name': 'Sugar', 'value': '34'}], 'name': 'Pauls - Smarter White Milk, 600 ml'}, {'nutritions': [{'nam

# მოდელისთვის მომზადება

<span style="font-size: 16px;">ვინაიდან მომხმარებლის მომდევნო საკვების გასარკვევად მნიშვნელოვანია რომ მაგალითები ყოველდღიური და ჯანსაღი დიეტებიდან ავიღოთ, საჭიროა, რომ დიეტების ხანგრძლივობა არ იყოს ხანმოკლე. მოდელისთვის მოსამზადებლად თავიდაპირველად მონაცემებიდან ვნახოთ, თუ რომელი მომხმარებლის ჩანაწერები (დიეტები) არის ზუსტად 35 დღის, ან უფრო მეტი ხანგრძლივობის (35 დღეზე ხანრგრძლივი ჩანაწერებიდან გამოვიყენებთ მხოლოდ პირველ 35 დღეს).</span>

In [55]:
# Minimum needed amount of days for the user's diet
NEEDED_DAYS_FOR_DIET = 35

# We will keep track of all the users whose diets' days 
# are less than needed amount in this array
invalid_users = []

days_count = 0
all_users = 0
valid_diets_count = 0
valid_diets_days_count = 0
curr_user_id = train.iloc[0, 0]
for i in range(len(train.index)):
    # Get the details of the day for the current row
    curr_day_user_id = train.iloc[i, 0]

    if curr_day_user_id != curr_user_id:
        all_users += 1
        if days_count >= NEEDED_DAYS_FOR_DIET:
            valid_diets_count += 1
            valid_diets_days_count += days_count
        else:
            invalid_users.append(curr_user_id)
        curr_user_id = curr_day_user_id
        days_count = 0 
    days_count += 1

# Last check after the loop
if days_count >= NEEDED_DAYS_FOR_DIET:
    valid_diets_count += 1
    valid_diets_days_count += days_count
else:
    invalid_users.append(curr_user_id)
    
print('{0} მომხმარებლიდან {1} დღეზე ხანგრძლივი დიეტის ჩანაწერი არის {2} მომხმარებელზე'.format(
    all_users, NEEDED_DAYS_FOR_DIET, valid_diets_count))
print('აღნიშნულ მომხმარებლების საერთო დიეტის დღეების რაოდენობა კი ' + str(valid_diets_days_count) + '-ია')

9895 მომხმარებლიდან 35 დღეზე ხანგრძლივი დიეტის ჩანაწერი არის 5425 მომხმარებელზე
აღნიშნულ მომხმარებლების საერთო დიეტის დღეების რაოდენობა კი 527788-ია


<span style="font-size: 16px;">ახლა კი წავშალოთ იმ მომხმარებლების მონაცემები, რომელთა დიეტის ხანგრძლივობაც 35 დღეზე ნაკლებია. მიღებული მონაცემები შევინახოთ `valid_days_trained` ცვლადში.</span>

In [56]:
# First we make the copy of the original data
valid_days_train = train.copy()

# And now we drop the rows for the invalid users
i = 0
rows_to_drop = []
while i < len(valid_days_train.index):
    curr_user_id = valid_days_train.iloc[i, 0]
    if curr_user_id in invalid_users:
        rows_to_drop.append(i)
    i += 1
    
valid_days_train = valid_days_train.drop(rows_to_drop)

527788


<span style="font-size: 16px;">**!!!!!!!!!!ეს მერე!!!!!!!!!!1** შემდეგ ნაწილში ვნახოთ, თუ რომელი მომხმარებლების დიეტები არის ჩვენთვის ვალიდური (არ არის დარღვეული დიდი პროცენტობით) და დავტოვოთ მოხლოდ მათი ჩანაწერები</span>